In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 8

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    xe1 = torch.rand(Nb, dimension).cuda()  
    xe1[:, 4] = torch.zeros(Nb)
    xe2 = torch.rand(Nb, dimension).cuda()  
    xe2[:, 4] = torch.ones(Nb)
    xf1 = torch.rand(Nb, dimension).cuda() 
    xf1[:, 5] = torch.zeros(Nb)
    xf2 = torch.rand(Nb, dimension).cuda() 
    xf2[:, 5] = torch.ones(Nb)
    xg1 = torch.rand(Nb, dimension).cuda() 
    xg1[:, 6] = torch.zeros(Nb)
    xg2 = torch.rand(Nb, dimension).cuda() 
    xg2[:, 6] = torch.ones(Nb)
    xh1 = torch.rand(Nb, dimension).cuda() 
    xh1[:, 7] = torch.zeros(Nb)
    xh2 = torch.rand(Nb, dimension).cuda() 
    xh2[:, 7] = torch.ones(Nb)
    
    sa1 = (torch.sum((model(xa1)-u_ex(xa1))**2)/xa1.size()[0]).cuda() 
    sa2 = (torch.sum((model(xa2)-u_ex(xa2))**2)/xa1.size()[0]).cuda()  
    sb1 = (torch.sum((model(xb1)-u_ex(xb1))**2)/xa1.size()[0]).cuda()  
    sb2 = (torch.sum((model(xb2)-u_ex(xb2))**2)/xa1.size()[0]).cuda() 
    sc1 = (torch.sum((model(xc1)-u_ex(xc1))**2)/xa1.size()[0]).cuda()  
    sc2 = (torch.sum((model(xc2)-u_ex(xc2))**2)/xa1.size()[0]).cuda() 
    sd1 = (torch.sum((model(xd1)-u_ex(xd1))**2)/xa1.size()[0]).cuda()  
    sd2 = (torch.sum((model(xd2)-u_ex(xd2))**2)/xa1.size()[0]).cuda() 
    se1 = (torch.sum((model(xe1)-u_ex(xe1))**2)/xa1.size()[0]).cuda() 
    se2 = (torch.sum((model(xe2)-u_ex(xe2))**2)/xa1.size()[0]).cuda()  
    sf1 = (torch.sum((model(xf1)-u_ex(xf1))**2)/xa1.size()[0]).cuda()  
    sf2 = (torch.sum((model(xf2)-u_ex(xf2))**2)/xa1.size()[0]).cuda() 
    sg1 = (torch.sum((model(xg1)-u_ex(xg1))**2)/xa1.size()[0]).cuda()  
    sg2 = (torch.sum((model(xg2)-u_ex(xg2))**2)/xa1.size()[0]).cuda() 
    sh1 = (torch.sum((model(xh1)-u_ex(xh1))**2)/xa1.size()[0]).cuda()  
    sh2 = (torch.sum((model(xh2)-u_ex(xh2))**2)/xa1.size()[0]).cuda()
    
    part_2 = sa1 + sa2 + sb1 + sb2 + sc1 + sc2 + sd1 + sd2 + se1 + se2 + sf1 + sf2 + sg1 + sg2 + sh1 + sh2
    
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 / 16

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 16)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=8, out_features=16, bias=True)
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=16, bias=True)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=16, bias=True)
  (linear5): Linear(in_features=16, out_features=16, bias=True)
  (linear6): Linear(in_features=16, out_features=16, bias=True)
  (linear_out): Linear(in_features=16, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 50000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_8D_Dirichlet.npy", error_save)
np.save("DRM_relative_error_8D_Dirichlet.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(14724.2067, device='cuda:1')
current relative error is:  tensor(5.2812, device='cuda:1')
current epoch is:  50
current loss is:  tensor(604.9467, device='cuda:1')
current relative error is:  tensor(1.1012, device='cuda:1')
current epoch is:  100
current loss is:  tensor(352.5237, device='cuda:1')
current relative error is:  tensor(0.8730, device='cuda:1')
current epoch is:  150
current loss is:  tensor(215.0346, device='cuda:1')
current relative error is:  tensor(0.6770, device='cuda:1')
current epoch is:  200
current loss is:  tensor(113.2547, device='cuda:1')
current relative error is:  tensor(0.5090, device='cuda:1')
current epoch is:  250
current loss is:  tensor(54.5639, device='cuda:1')
current relative error is:  tensor(0.3896, device='cuda:1')
current epoch is:  300
current loss is:  tensor(37.8045, device='cuda:1')
current relative error is:  tensor(0.3299, device='cuda:1')
current epoch is:  350
current loss is:  tensor(18.0173, d

current epoch is:  3050
current loss is:  tensor(-29.2534, device='cuda:1')
current relative error is:  tensor(0.1091, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-28.8796, device='cuda:1')
current relative error is:  tensor(0.1115, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-30.0957, device='cuda:1')
current relative error is:  tensor(0.1121, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-28.7001, device='cuda:1')
current relative error is:  tensor(0.1119, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-31.5018, device='cuda:1')
current relative error is:  tensor(0.1077, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-25.2337, device='cuda:1')
current relative error is:  tensor(0.1083, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-30.5105, device='cuda:1')
current relative error is:  tensor(0.1082, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-36.9443, device='cuda:1')
current relative error is:  tensor(0.0773, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-34.5595, device='cuda:1')
current relative error is:  tensor(0.0727, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-35.2354, device='cuda:1')
current relative error is:  tensor(0.0772, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-35.0853, device='cuda:1')
current relative error is:  tensor(0.0714, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-35.5664, device='cuda:1')
current relative error is:  tensor(0.0704, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-34.3515, device='cuda:1')
current relative error is:  tensor(0.0737, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-33.5985, device='cuda:1')
current relative error is:  tensor(0.0703, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-38.4520, device='cuda:1')
current relative error is:  tensor(0.0357, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-36.7612, device='cuda:1')
current relative error is:  tensor(0.0337, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-35.5678, device='cuda:1')
current relative error is:  tensor(0.0601, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-39.4273, device='cuda:1')
current relative error is:  tensor(0.0366, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-39.3440, device='cuda:1')
current relative error is:  tensor(0.0357, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-37.6567, device='cuda:1')
current relative error is:  tensor(0.0387, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-36.0136, device='cuda:1')
current relative error is:  tensor(0.0360, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-38.5198, device='cuda:1')
current relative error is:  tensor(0.0258, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-38.2802, device='cuda:1')
current relative error is:  tensor(0.0484, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-42.2528, device='cuda:1')
current relative error is:  tensor(0.0325, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-38.2854, device='cuda:1')
current relative error is:  tensor(0.0254, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-37.9511, device='cuda:1')
current relative error is:  tensor(0.0265, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-39.6893, device='cuda:1')
current relative error is:  tensor(0.0302, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-40.3443, device='cuda:1')
current relative error is:  tensor(0.0301, device='cuda:1')
current epoch is:  12500
current loss is:

...

current epoch is:  36150
current loss is:  tensor(-41.2830, device='cuda:1')
current relative error is:  tensor(0.0195, device='cuda:1')
current epoch is:  36200
current loss is:  tensor(-38.4693, device='cuda:1')
current relative error is:  tensor(0.0128, device='cuda:1')
current epoch is:  36250
current loss is:  tensor(-39.7200, device='cuda:1')
current relative error is:  tensor(0.0132, device='cuda:1')
current epoch is:  36300
current loss is:  tensor(-38.1669, device='cuda:1')
current relative error is:  tensor(0.0129, device='cuda:1')
current epoch is:  36350
current loss is:  tensor(-38.4613, device='cuda:1')
current relative error is:  tensor(0.0214, device='cuda:1')
current epoch is:  36400
current loss is:  tensor(-37.8880, device='cuda:1')
current relative error is:  tensor(0.0174, device='cuda:1')
current epoch is:  36450
current loss is:  tensor(-38.5231, device='cuda:1')
current relative error is:  tensor(0.0284, device='cuda:1')
current epoch is:  36500
current loss is:

current epoch is:  39150
current loss is:  tensor(-39.6854, device='cuda:1')
current relative error is:  tensor(0.0137, device='cuda:1')
current epoch is:  39200
current loss is:  tensor(-38.1035, device='cuda:1')
current relative error is:  tensor(0.0173, device='cuda:1')
current epoch is:  39250
current loss is:  tensor(-40.6438, device='cuda:1')
current relative error is:  tensor(0.0152, device='cuda:1')
current epoch is:  39300
current loss is:  tensor(-38.8243, device='cuda:1')
current relative error is:  tensor(0.0190, device='cuda:1')
current epoch is:  39350
current loss is:  tensor(-41.3964, device='cuda:1')
current relative error is:  tensor(0.0139, device='cuda:1')
current epoch is:  39400
current loss is:  tensor(-40.6488, device='cuda:1')
current relative error is:  tensor(0.0149, device='cuda:1')
current epoch is:  39450
current loss is:  tensor(-38.5177, device='cuda:1')
current relative error is:  tensor(0.0354, device='cuda:1')
current epoch is:  39500
current loss is:

current epoch is:  42150
current loss is:  tensor(-41.5124, device='cuda:1')
current relative error is:  tensor(0.0152, device='cuda:1')
current epoch is:  42200
current loss is:  tensor(-41.3297, device='cuda:1')
current relative error is:  tensor(0.0251, device='cuda:1')
current epoch is:  42250
current loss is:  tensor(-36.8289, device='cuda:1')
current relative error is:  tensor(0.0128, device='cuda:1')
current epoch is:  42300
current loss is:  tensor(-38.0009, device='cuda:1')
current relative error is:  tensor(0.0306, device='cuda:1')
current epoch is:  42350
current loss is:  tensor(-38.3941, device='cuda:1')
current relative error is:  tensor(0.0127, device='cuda:1')
current epoch is:  42400
current loss is:  tensor(-41.5179, device='cuda:1')
current relative error is:  tensor(0.0142, device='cuda:1')
current epoch is:  42450
current loss is:  tensor(-37.1274, device='cuda:1')
current relative error is:  tensor(0.0117, device='cuda:1')
current epoch is:  42500
current loss is:

current epoch is:  45150
current loss is:  tensor(-40.7991, device='cuda:1')
current relative error is:  tensor(0.0145, device='cuda:1')
current epoch is:  45200
current loss is:  tensor(-37.5219, device='cuda:1')
current relative error is:  tensor(0.0173, device='cuda:1')
current epoch is:  45250
current loss is:  tensor(-35.7846, device='cuda:1')
current relative error is:  tensor(0.0117, device='cuda:1')
current epoch is:  45300
current loss is:  tensor(-36.3700, device='cuda:1')
current relative error is:  tensor(0.0153, device='cuda:1')
current epoch is:  45350
current loss is:  tensor(-41.3898, device='cuda:1')
current relative error is:  tensor(0.0127, device='cuda:1')
current epoch is:  45400
current loss is:  tensor(-36.7566, device='cuda:1')
current relative error is:  tensor(0.0118, device='cuda:1')
current epoch is:  45450
current loss is:  tensor(-39.3340, device='cuda:1')
current relative error is:  tensor(0.0166, device='cuda:1')
current epoch is:  45500
current loss is:

current epoch is:  48150
current loss is:  tensor(-40.6028, device='cuda:1')
current relative error is:  tensor(0.0218, device='cuda:1')
current epoch is:  48200
current loss is:  tensor(-38.3725, device='cuda:1')
current relative error is:  tensor(0.0230, device='cuda:1')
current epoch is:  48250
current loss is:  tensor(-39.8729, device='cuda:1')
current relative error is:  tensor(0.0140, device='cuda:1')
current epoch is:  48300
current loss is:  tensor(-41.6305, device='cuda:1')
current relative error is:  tensor(0.0195, device='cuda:1')
current epoch is:  48350
current loss is:  tensor(-43.3559, device='cuda:1')
current relative error is:  tensor(0.0108, device='cuda:1')
current epoch is:  48400
current loss is:  tensor(-39.2264, device='cuda:1')
current relative error is:  tensor(0.0182, device='cuda:1')
current epoch is:  48450
current loss is:  tensor(-40.5802, device='cuda:1')
current relative error is:  tensor(0.0110, device='cuda:1')
current epoch is:  48500
current loss is:

In [18]:
x_test = torch.rand(10, 8)
torch.abs(model(x) - u_ex(x))

tensor([[0.0025],
        [0.0085],
        [0.0166],
        ...,
        [0.0029],
        [0.0181],
        [0.0407]], device='cuda:1', grad_fn=<AbsBackward>)